In [1]:
import re

In [2]:
file = open("en1.8m.txt", "r", encoding="utf8")
whole_text = file.read()
text = whole_text.split('\n')
test = text[0]
test

"Greek Christian scribes played a crucial role in the preservation of Aristotle by copying all the extant Greek language manuscripts of the corpus. The first Greek Christians to comment extensively on Aristotle were John Philoponus, Elias, and David in the sixth century, and Stephen of Alexandria in the early seventh century. John Philoponus stands out for having attempted a fundamental critique of Aristotle's views on the eternity of the world, movement, and other elements of Aristotelian thought. After a hiatus of several centuries, formal commentary by Eustratius and Michael of Ephesus reappears in the late eleventh and early twelfth centuries, apparently sponsored by Anna Comnena."

In [3]:
#whole_text = whole_text.replace('\n', '.')

In [4]:
def append_space(text, pattern, left=True):
    p = re.compile("[" + pattern + "]")
    
    for i, m in enumerate(p.finditer(text)):
        if left:
            text = text[: m.start() + i] + " " + text[m.start() + i :]
        else:
            text = text[: m.start() + 1 + i] + " " + text[m.start() + 1 + i :]
            
    return text

print(append_space(test, ",."))

Greek Christian scribes played a crucial role in the preservation of Aristotle by copying all the extant Greek language manuscripts of the corpus . The first Greek Christians to comment extensively on Aristotle were John Philoponus , Elias , and David in the sixth century , and Stephen of Alexandria in the early seventh century . John Philoponus stands out for having attempted a fundamental critique of Aristotle's views on the eternity of the world , movement , and other elements of Aristotelian thought . After a hiatus of several centuries , formal commentary by Eustratius and Michael of Ephesus reappears in the late eleventh and early twelfth centuries , apparently sponsored by Anna Comnena .


In [5]:
# dictionary = dict()

# for i in range(len(text)):
#     text[i] = append_space(text[i], ",\])>")
#     text[i] = append_space(text[i], ".")
#     text[i] = append_space(text[i], "\[(<", False)
    
#     for w in text[i].split():
#         if w.isalpha():
#             try:
#                 dictionary[w.lower()] += 1
#             except KeyError:
#                 dictionary[w.lower()] = 1

In [6]:
# it = 0

# for i in sorted(dictionary, key=dictionary.get, reverse=True): 
#     if it > 100:
#         break

#     print((i, dictionary[i]), end =" ")
#     it += 1

In [7]:
for i in range(len(text)):
    text[i] = append_space(text[i], ",\])>")
    text[i] = append_space(text[i], ".")
    text[i] = append_space(text[i], "\[(<", False)

In [8]:
def build_n_gram(text_arr, n_gram, grams):
    cur_gram = []
    count_gram = 0
#     grams = dict()

    for i in range(len(text_arr)):
        if not text_arr[i].isalpha():
            continue

        cur_gram = [text_arr[i]]

        for j in range(i + 1, i + n_gram):
            if j < len(text_arr):
                if not text_arr[j].isalpha():
                    break

                cur_gram.append(text_arr[j])

        if len(cur_gram) == n_gram:
            try:
                grams[" ".join(cur_gram)] += 1
            except KeyError:
                grams[" ".join(cur_gram)] = 1

    return grams

In [9]:
def build_dict1(text, n_s):
    dictionary = []

    for n in range(n_s):
        dictionary.append(build_n_gram(text.split(), n + 1))
        
    return dictionary

In [10]:
def build_dict(text, n_s):
    dictionary = []
    for n in range(n_s):
        dictionary.append(dict())
    
    for n in range(n_s):
        for line in text:
            dictionary[n] = build_n_gram(line.split(), n + 1, dictionary[n])
        
    return dictionary

In [11]:
def SCP_f(words, dictionary):
    words_arr = words.split()
    
    numerator = dictionary[len(words_arr) - 1][words] ** 2
    F = 1 / (len(words_arr) - 1)
    denominator = 0
    
    for i in range(len(words_arr) - 1):
        left = words_arr[0:i+1]
        right = words_arr[i+1:]
        
        denominator += dictionary[len(left) - 1][" ".join(left)] * dictionary[len(right) - 1][" ".join(right)]
        
    F *= denominator
    return numerator / F

In [12]:
# print(SCP_f('Greek Christian', big_dict))
# print(SCP_f('Anna Comnena', big_dict))

In [21]:
def SCP_f_n(dictionary, table, n):
    for s in dictionary[n - 1]:
        table[n - 2][s] = SCP_f(s, dictionary)

In [25]:
def main(text, max_gram):
    dictionary = build_dict(text, max_gram)
    #print(dictionary[0])
    
    table = []
    for i in range(max_gram - 1):
        table.append(dict())

    for i in range(2, max_gram + 1):
        print("\n")
        print(i, "grams")
        SCP_f_n(dictionary, table, i)
        
    return table

In [26]:
table = main(text, 3)
table



2 grams


3 grams


[{'Greek Christian': 7.186489399928136e-05,
  'Christian scribes': 0.003952569169960474,
  'scribes played': 0.0009250693802035153,
  'played a': 0.00013663233722573538,
  'a crucial': 0.00012472283813747228,
  'crucial role': 0.0023148148148148147,
  'role in': 0.001127392457399432,
  'in the': 0.02845781222747467,
  'the preservation': 2.4058026034152294e-05,
  'preservation of': 7.297366384789698e-05,
  'of Aristotle': 2.1301976823449217e-05,
  'Aristotle by': 6.68645858407553e-06,
  'by copying': 2.005937575222659e-05,
  'copying all': 0.0001375894331315355,
  'all the': 0.00017314999598013158,
  'the extant': 9.494900941478772e-06,
  'extant Greek': 0.00030303030303030303,
  'Greek language': 0.00012945914844649022,
  'language manuscripts': 0.00012658227848101267,
  'manuscripts of': 2.556237218813906e-06,
  'of the': 0.04837857955808627,
  'the corpus': 1.2716385189480497e-06,
  'The first': 0.0018035487293031325,
  'first Greek': 1.2186205215695831e-06,
  'Greek Christians': 0.